In [ ]:
import numpy as np
import scipy.optimize as scyopt
import matplotlib.pyplot as plt
import cProfile

import pyaccel, sirius
import mathphys as mp
%load_ext autoreload
%autoreload 1
%aimport lattice_errors

In [ ]:
#import sirius.SI_V10 as v10
#acc      = v10.create_accelerator()
#fam_data = v10.get_family_data(acc)

acc      = sirius.SI_V12.create_accelerator()
fam_data = sirius.SI_V12.get_family_data(acc)

In [ ]:
um, mrad, percent = 1e-6, 1e-3, 1e-2
config = dict({'mags':dict(),'girder':dict()})
mags = []
dics = dict()
dics['labels'] = ['qfa','qda','qdb1','qdb2','qfb','qf1','qf2','qf3','qf4']
dics['x']     = 40 * um * 1
dics['y']     = 40 * um * 1
dics['roll']  = 0.30 * mrad * 1
dics['excit'] = 0.05 * percent * 1
mags.append(dics)

dics = dict()
dics['labels']   = ['sfa','sda','sf1j','sd1j','sd2j','sd3j','sf2j',
                    'sdb','sfb','sf1k','sd1k','sd2k','sd3k','sf2k']
dics['x']     = 40 * um * 1
dics['y']     = 40 * um * 1
dics['roll']  = 0.30 * mrad * 1
dics['excit'] = 0.05 * percent * 1
mags.append(dics)

dics = dict()
dics['labels']= ['bc_lf','bc_hf']
dics['x']     = 40 * um * 1
dics['y']     = 40 * um * 1
dics['roll']  = 0.30 * mrad * 1
dics['excit'] = 0.05 * percent * 1
dics['k_dip'] = 0.10 * percent * 1
mags.append(dics)

dics = dict()
dics['labels'] = ['b1','b2']
dics['nrsegs'] = [1,1]
dics['x']      = 40 * um * 1
dics['y']      = 40 * um * 1
dics['roll']   = 0.30 * mrad * 1
dics['excit']  = 0.05 * percent * 1
dics['k_dip']  = 0.10 * percent * 1
mags.append(dics)

girder = dict()
girder['x']    = 80 * um * 1
girder['y']    = 80 * um * 1
girder['roll'] = 0.3 * mrad * 1

errors = lattice_errors.generate_errors(acc,mags,girder,fam_data=fam_data,nr_mach=20,cutoff=1,rndtype='gauss',seed=19880419)

In [ ]:
machine = lattice_errors.apply_erros(acc,errors,increment=1.0)

In [ ]:
r = lattice_errors.calc_respm_cod(acc,
                                  fam_data['bpm']['index'],
                                  fam_data['ch']['index'],
                                  fam_data['cv']['index'],
                                  symmetry=10)

bba_ind = lattice_errors.get_bba_ind(acc,fam_data['bpm']['index'])

cor_conf = dict({'bpms':fam_data['bpm']['index'],
                 'hcms':fam_data['ch']['index'],
                 'vcms':fam_data['cv']['index'],
                 'bpm_err':dict({'sigma':(20e-6,20e-6),'cutoff':1})})
machine2 = [machine[i][:] for i in range(len(machine))]
gcodx, gcody = lattice_errors.correct_cod(machine2,ind_bba=bba_ind,sext_ramp=[0,1],respm=r,**cor_conf)